<a href="https://colab.research.google.com/github/henryjhu/Anomaly-Detection-in-Wire-Activities/blob/main/DSC_680_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DSC-680-Z1 Research Practicum** <BR> Machine Learning

## **Project Description**
A global bank sought to find new and innovative means for detecting and preventing fraud in their wire transactions. Their goal is through machine learning to arrive at scenario detection rules which could be customized with parameters specific for each cohort of customers. The immediate challenge was that the data provided had no labels. Therefore, unsupervised machine learning techniques were first employed for this research project. After the labels have been identified, supervised machine learning techniques were then employed with selected tuning parameters to increase both the sensitivity and specificity percentages of both classes.

<b>Purpose:</b>
Carry out supervised machine learnings with the sample data.<br>
<b>Universtiy Name:</b> Utica College <br>
<b>Course Name:</b> DSC-680-Z1 Research Practicum <br>
<b>Student Name:</b> Henry J. Hu <br>
<b>Program Director Name:</b> Dr. McCarthy, Michael <br>
<b>Runtime Environment:</b> Google Colab<br>
<b>Programming Language:</b> Python <br>
<b>Sample Data Frame:</b>
Six random samples of labeled international wires belonging to 139 customers from 3 continents for the entire year of 2020.<br>
<b> Last Update:</b> August 7th, 2021

## **Installing Packages Into Google Colab**

In [ ]:
!pip install -U imbalanced-learn

     |████████████████████████████████| 206 kB 8.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


## **Mounting Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


## **Importing Libraries**

In [ ]:
# Importing libraries
import io
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import sklearn
import traceback
import time
import pytz
import xgboost as xgb
import sklearn.exceptions
import warnings
from datetime import datetime
from sklearn import metrics
from scipy.stats import rankdata
from numpy import quantile, where, random
from random import sample
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.datasets import make_blobs
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, IsolationForest, VotingClassifier, StackingClassifier)
from imblearn.over_sampling import SMOTE # https://github.com/vsmolyakov/experiments_with_python/blob/master/chp01/imbalanced_data.ipynb

## **Importing Data into Google Colab**

In [ ]:
# Importing data
NN_103_output_df = pd.read_csv("gdrive/MyDrive/DSC-680/NN_103_score_df.csv")
NN_202_output_df = pd.read_csv("gdrive/MyDrive/DSC-680/NN_202_score_df.csv")
EU_103_output_df = pd.read_csv("gdrive/MyDrive/DSC-680/EU_103_score_df.csv")
EU_202_output_df = pd.read_csv("gdrive/MyDrive/DSC-680/EU_202_score_df.csv")
AS_103_output_df = pd.read_csv("gdrive/MyDrive/DSC-680/AS_103_score_df.csv")
AS_202_output_df = pd.read_csv("gdrive/MyDrive/DSC-680/AS_202_score_df.csv")

## **Re-Sampling**

In [ ]:
NN_103_output_df = NN_103_output_df.sample(n=100000, random_state=42)
NN_202_output_df = NN_202_output_df.sample(n=100000, random_state=42)
EU_103_output_df = EU_103_output_df.sample(n=100000, random_state=42)
EU_202_output_df = EU_202_output_df.sample(n=100000, random_state=42)
AS_103_output_df = AS_103_output_df.sample(n=100000, random_state=42)
AS_202_output_df = AS_202_output_df.sample(n=200000, random_state=42)

## **Preparation of Training and Test Data Sets**

In [ ]:
NN_103_output_df_x_train, NN_103_output_df_x_test, NN_103_output_df_y_train, NN_103_output_df_y_test = train_test_split(NN_103_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],NN_103_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
NN_202_output_df_x_train, NN_202_output_df_x_test, NN_202_output_df_y_train, NN_202_output_df_y_test = train_test_split(NN_202_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],NN_202_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
EU_103_output_df_x_train, EU_103_output_df_x_test, EU_103_output_df_y_train, EU_103_output_df_y_test = train_test_split(EU_103_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],EU_103_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
EU_202_output_df_x_train, EU_202_output_df_x_test, EU_202_output_df_y_train, EU_202_output_df_y_test = train_test_split(EU_202_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],EU_202_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
AS_103_output_df_x_train, AS_103_output_df_x_test, AS_103_output_df_y_train, AS_103_output_df_y_test = train_test_split(AS_103_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],AS_103_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
AS_202_output_df_x_train, AS_202_output_df_x_test, AS_202_output_df_y_train, AS_202_output_df_y_test = train_test_split(AS_202_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],AS_202_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42, stratify=AS_202_output_df[['FRAUD_LABEL']] )

## **Over Sampling the Minority Class Using SMOTE**

In [ ]:
# Initilize SMOTE
sm = SMOTE(random_state=42, n_jobs=-1)

# Apply SMOTE to North America
NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train = sm.fit_resample(NN_103_output_df_x_train, NN_103_output_df_y_train)
NN_103_sm_output_df_x_test, NN_103_sm_output_df_y_test = sm.fit_resample(NN_103_output_df_x_test, NN_103_output_df_y_test)
NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train = sm.fit_resample(NN_202_output_df_x_train, NN_202_output_df_y_train)
NN_202_sm_output_df_x_test, NN_202_sm_output_df_y_test = sm.fit_resample(NN_202_output_df_x_test, NN_202_output_df_y_test)

#Apply SMOTE to Europe
EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train = sm.fit_resample(EU_103_output_df_x_train, EU_103_output_df_y_train)
EU_103_sm_output_df_x_test, EU_103_sm_output_df_y_test = sm.fit_resample(EU_103_output_df_x_test, EU_103_output_df_y_test)
EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train = sm.fit_resample(EU_202_output_df_x_train, EU_202_output_df_y_train)
EU_202_sm_output_df_x_test, EU_202_sm_output_df_y_test = sm.fit_resample(EU_202_output_df_x_test, EU_202_output_df_y_test)

#Apply SMOTE to Asia
AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train = sm.fit_resample(AS_103_output_df_x_train, AS_103_output_df_y_train)
AS_103_sm_output_df_x_test, AS_103_sm_output_df_y_test = sm.fit_resample(AS_103_output_df_x_test, AS_103_output_df_y_test)
AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train = sm.fit_resample(AS_202_output_df_x_train, AS_202_output_df_y_train)
AS_202_sm_output_df_x_test, AS_202_sm_output_df_y_test = sm.fit_resample(AS_202_output_df_x_test, AS_202_output_df_y_test)

# unique, count = np.unique(NN_103_sm_output_df_y_train, return_counts=True)
# y_train_ct = {k:v for (k,v) in zip(unique, count)}
# y_train_ct
# NN_103_sm_output_df_x_train

## **Cross Validation Function**

In [ ]:
##############################################################################################
#
# Purpose: 
# Through cross validation, performs model fitting, training and prediction for 
# Random Forest and Artificial Neuro Networks 
#
##############################################################################################

def cross_val_train (clf_n, X_train, Y_train):

  # Initialize the log variable
  class log:
    def_tz = pytz.timezone('America/New_York')
    def info(text):        
        print(f'{datetime.now(log.def_tz).replace(microsecond=0)} : {text}');

  # Y train has to be a 1D array
  Y_train = np.ravel(Y_train)

  # Scale and center the data around the mean of 0
  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)

  # Random Seed
  random.seed(42)

  clf1=MLPClassifier()
  clf2=RandomForestClassifier()

  # Best tunning hyper parameters have been determined with two random iterations
  # Data frames used: NN_202_output_df_x_train, NN_202_output_df_y_train
  clf1_parameter_space = {
      'warm_start': [True],
      'max_iter': [5000, 10000, 20000],
      # 'max_iter': [10000],
      'hidden_layer_sizes': [(4,8),(8,16),(16,32)],
      # 'hidden_layer_sizes': [(8,16)],
      'activation': ['logistic'],
      'solver': ['lbfgs'], # Have to set learning_rate_init if solver is either ’sgd’ or ‘adam’ 
      'alpha': [0.05,0.1,1.5,2.0,2.5, 5,10,25], # Have to set alpha correctly or else all same label values in output 
      # 'alpha': [2.0],
      'learning_rate': ['adaptive'], # No need to set learning_rate if solver is not ’sgd’
      'random_state': [42]
  }
  # Best tunning hyper parameters have been determined with two random iterations
  # Data frames used: NN_202_output_df_x_train, NN_202_output_df_y_train
  clf2_parameter_space = {
       'warm_start': [True],
      'bootstrap': [True, False],
      # 'bootstrap': [False],
      'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100,300,500, None],
      # 'max_depth': [70],
      'max_features': ['auto', 'sqrt'],
      # 'max_features': ['sqrt'],
      'min_samples_leaf': [1, 2, 4,5,10,20,30,40,50],
      # 'min_samples_leaf': [1],
      'min_samples_split': [2, 5, 10,20,25,30],
      # 'min_samples_split': [10],
      'n_estimators': [200, 400, 500, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 3000, 5000]
      # 'n_estimators': [1000]
  }
  
  if clf_n == 1:
    model = clf1
    model_name = 'Artificial Neuro Networks'
    clf_parameter_space = clf1_parameter_space
  else: 
    model = clf2
    model_name = 'Random Forest'
    clf_parameter_space = clf2_parameter_space

  clf_cross = RandomizedSearchCV(model, clf_parameter_space, cv=10, n_iter=12, verbose=True, refit=True) # Stratified cv fold cross validation
  # clf_cross = RandomizedSearchCV(model, clf_parameter_space, cv=10, n_iter=1, verbose=True, refit=True) # Stratified cv fold cross validation
  clf_cross.fit(X_train, Y_train)

  log.info(f'Best parameters used for cross validation of {model_name}')
  log.info(clf_cross.best_params_)

  return clf_cross

## **Stacking - Level 1**

### **Gathering Predictions for Test Data Set X**

### ***Artificial Neural Network Predictions***

#### ***North America***

In [ ]:
NN_103_sm_output_ann_y_pred_test = cross_val_train(1,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_test)
NN_103_sm_output_ann_y_pred_train = cross_val_train(1,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 17:26:49-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 17:26:49-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 17:28:25-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 17:28:25-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}


In [ ]:
NN_202_sm_output_ann_y_pred_test = cross_val_train(1,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_test)
NN_202_sm_output_ann_y_pred_train = cross_val_train(1,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 17:31:57-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 17:31:57-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 17:35:29-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 17:35:29-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}


#### ***Europe***

In [ ]:
EU_103_sm_output_ann_y_pred_test = cross_val_train(1, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_test)
EU_103_sm_output_ann_y_pred_train = cross_val_train(1, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 20:37:28-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 20:37:28-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 20:39:11-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 20:39:11-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}


In [ ]:
EU_202_sm_output_ann_y_pred_test = cross_val_train(1, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_test)
EU_202_sm_output_ann_y_pred_train = cross_val_train(1, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 17:43:50-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 17:43:50-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 17:48:51-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-11 17:48:51-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 2.0, 'activation': 'logistic'}


#### ***Asia***

In [ ]:
AS_103_sm_output_ann_y_pred_test = cross_val_train(1, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_test)
AS_103_sm_output_ann_y_pred_train = cross_val_train(1, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-14 17:57:36-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-14 17:57:36-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 20000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 16), 'alpha': 25.0, 'activation': 'logistic'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-14 18:01:01-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-14 18:01:01-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 20000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 16), 'alpha': 25.0, 'activation': 'logistic'}


In [ ]:
AS_202_sm_output_ann_y_pred_test = cross_val_train(1, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_test)
AS_202_sm_output_ann_y_pred_train = cross_val_train(1, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-23 02:30:16-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-23 02:30:16-04:00 : {'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 16), 'alpha': 2.0, 'activation': 'logistic'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-23 02:46:42-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-23 02:46:42-04:00 : {'solver': 'lbfgs', 'random_state': 42, 'max_iter': 10000, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 16), 'alpha': 2.0, 'activation': 'logistic'}


### ***Random Forest Predictions***

#### ***North America***

In [ ]:
NN_103_sm_output_rf_y_pred_test = cross_val_train(2,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_test)
NN_103_sm_output_rf_y_pred_train = cross_val_train(2,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 21:05:31-04:00 : Best parameters used for cross validation of Random Forest
2021-08-11 21:05:31-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 21:18:04-04:00 : Best parameters used for cross validation of Random Forest
2021-08-11 21:18:04-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}


In [ ]:
NN_202_sm_output_rf_y_pred_test = cross_val_train(2,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_test)
NN_202_sm_output_rf_y_pred_train = cross_val_train(2,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 23:56:42-04:00 : Best parameters used for cross validation of Random Forest
2021-08-11 23:56:42-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-12 00:09:54-04:00 : Best parameters used for cross validation of Random Forest
2021-08-12 00:09:54-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}


#### ***Europe***

In [ ]:
EU_103_sm_output_rf_y_pred_test = cross_val_train(2, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_test)
EU_103_sm_output_rf_y_pred_train = cross_val_train(2, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 21:56:21-04:00 : Best parameters used for cross validation of Random Forest
2021-08-11 21:56:21-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-11 22:07:36-04:00 : Best parameters used for cross validation of Random Forest
2021-08-11 22:07:36-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}


In [ ]:
EU_202_sm_output_rf_y_pred_test = cross_val_train(2, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_test)
EU_202_sm_output_rf_y_pred_train = cross_val_train(2, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-12 00:20:31-04:00 : Best parameters used for cross validation of Random Forest
2021-08-12 00:20:31-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-12 00:30:55-04:00 : Best parameters used for cross validation of Random Forest
2021-08-12 00:30:55-04:00 : {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}


#### ***Asia***

In [ ]:
AS_103_sm_output_rf_y_pred_test = cross_val_train(2, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_test)
AS_103_sm_output_rf_y_pred_train = cross_val_train(2, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-14 18:16:59-04:00 : Best parameters used for cross validation of Random Forest
2021-08-14 18:16:59-04:00 : {'warm_start': True, 'n_estimators': 500, 'min_samples_split': 25, 'min_samples_leaf': 50, 'max_features': 'sqrt', 'max_depth': 500, 'bootstrap': True}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-14 18:20:11-04:00 : Best parameters used for cross validation of Random Forest
2021-08-14 18:20:11-04:00 : {'warm_start': True, 'n_estimators': 500, 'min_samples_split': 25, 'min_samples_leaf': 50, 'max_features': 'sqrt', 'max_depth': 500, 'bootstrap': True}


In [ ]:
AS_202_sm_output_rf_y_pred_test = cross_val_train(2, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_test)
AS_202_sm_output_rf_y_pred_train = cross_val_train(2, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-23 03:10:11-04:00 : Best parameters used for cross validation of Random Forest
2021-08-23 03:10:11-04:00 : {'warm_start': True, 'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
2021-08-23 03:33:39-04:00 : Best parameters used for cross validation of Random Forest
2021-08-23 03:33:39-04:00 : {'warm_start': True, 'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}


## **Stacking - Level 2**

##### **Stacking Algorithm = XGBoost**

##### **Weights of 2x applied to Level 1 Random Forest's predictions before feeding them into Level 2 Decesion Tree as inputs**

##### **Level 2 X Training Data = Combine of Level 1 ANN and Random Forest predicitons from the "train" data**

##### **Level 2 X Test Data = Combine of Level 1 ANN and Random Forest predicitons from the "test" data**

#### ***North America***

In [ ]:
NN_103_sm_output_ensemble_x_train = np.column_stack((NN_103_sm_output_ann_y_pred_train, NN_103_sm_output_rf_y_pred_train*2))
NN_103_sm_output_ensemble_x_train_df = pd.DataFrame(NN_103_sm_output_ensemble_x_train, columns = ['ANN','RF'])
NN_103_sm_output_ensemble_x_test = np.column_stack((NN_103_sm_output_ann_y_pred_test, NN_103_sm_output_rf_y_pred_test*2))
NN_103_sm_output_ensemble_x_test_df = pd.DataFrame(NN_103_sm_output_ensemble_x_test, columns = ['ANN','RF'])
NN_103_sm_output_df_y_train = np.ravel(NN_103_sm_output_df_y_train)
NN_103_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(NN_103_sm_output_ensemble_x_train_df, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_ensemble_x_test_df)

In [ ]:
NN_202_sm_output_ensemble_x_train = np.column_stack((NN_202_sm_output_ann_y_pred_train, NN_202_sm_output_rf_y_pred_train*2))
NN_202_sm_output_ensemble_x_train_df = pd.DataFrame(NN_202_sm_output_ensemble_x_train, columns = ['ANN','RF'])
NN_202_sm_output_ensemble_x_test = np.column_stack((NN_202_sm_output_ann_y_pred_test, NN_202_sm_output_rf_y_pred_test*2))
NN_202_sm_output_ensemble_x_test_df = pd.DataFrame(NN_202_sm_output_ensemble_x_test, columns = ['ANN','RF'])
NN_202_sm_output_df_y_train = np.ravel(NN_202_sm_output_df_y_train)
NN_202_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(NN_202_sm_output_ensemble_x_train_df, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_ensemble_x_test_df)

#### ***Europe***

In [ ]:
EU_103_sm_output_ensemble_x_train = np.column_stack((EU_103_sm_output_ann_y_pred_train, EU_103_sm_output_rf_y_pred_train*2))
EU_103_sm_output_ensemble_x_train_df = pd.DataFrame(EU_103_sm_output_ensemble_x_train, columns = ['ANN','RF'])
EU_103_sm_output_ensemble_x_test = np.column_stack((EU_103_sm_output_ann_y_pred_test, EU_103_sm_output_rf_y_pred_test*2))
EU_103_sm_output_ensemble_x_test_df = pd.DataFrame(EU_103_sm_output_ensemble_x_test, columns = ['ANN','RF'])
EU_103_sm_output_df_y_train = np.ravel(EU_103_sm_output_df_y_train)
EU_103_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(EU_103_sm_output_ensemble_x_train_df, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_ensemble_x_test_df)

In [ ]:
EU_202_sm_output_ensemble_x_train = np.column_stack((EU_202_sm_output_ann_y_pred_train, EU_202_sm_output_rf_y_pred_train*2))
EU_202_sm_output_ensemble_x_train_df = pd.DataFrame(EU_202_sm_output_ensemble_x_train, columns = ['ANN','RF'])
EU_202_sm_output_ensemble_x_test = np.column_stack((EU_202_sm_output_ann_y_pred_test, EU_202_sm_output_rf_y_pred_test*2))
EU_202_sm_output_ensemble_x_test_df = pd.DataFrame(EU_202_sm_output_ensemble_x_test, columns = ['ANN','RF'])
EU_202_sm_output_df_y_train = np.ravel(EU_202_sm_output_df_y_train)
EU_202_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(EU_202_sm_output_ensemble_x_train_df, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_ensemble_x_test_df)

#### ***Asia***

In [ ]:
AS_103_sm_output_ensemble_x_train = np.column_stack((AS_103_sm_output_ann_y_pred_train, AS_103_sm_output_rf_y_pred_train*2))
AS_103_sm_output_ensemble_x_train_df = pd.DataFrame(AS_103_sm_output_ensemble_x_train, columns = ['ANN','RF'])
AS_103_sm_output_ensemble_x_test = np.column_stack((AS_103_sm_output_ann_y_pred_test, AS_103_sm_output_rf_y_pred_test*2))
AS_103_sm_output_ensemble_x_test_df = pd.DataFrame(AS_103_sm_output_ensemble_x_test, columns = ['ANN','RF'])
AS_103_sm_output_df_y_train = np.ravel(AS_103_sm_output_df_y_train)
AS_103_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(AS_103_sm_output_ensemble_x_train_df, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_ensemble_x_test_df)

In [ ]:
AS_202_sm_output_ensemble_x_train = np.column_stack((AS_202_sm_output_ann_y_pred_train, AS_202_sm_output_rf_y_pred_train*2))
AS_202_sm_output_ensemble_x_train_df = pd.DataFrame(AS_202_sm_output_ensemble_x_train, columns = ['ANN','RF'])
AS_202_sm_output_ensemble_x_test = np.column_stack((AS_202_sm_output_ann_y_pred_test, AS_202_sm_output_rf_y_pred_test*2))
AS_202_sm_output_ensemble_x_test_df = pd.DataFrame(AS_202_sm_output_ensemble_x_test, columns = ['ANN','RF'])
AS_202_sm_output_df_y_train = np.ravel(AS_202_sm_output_df_y_train)
AS_202_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(AS_202_sm_output_ensemble_x_train_df, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_ensemble_x_test_df)

## **Confusion Matricies of Accuracy Statistics**

**Accuracy** <br>
Accuracy is the most intuitive performance measure and it is simply a ratio of correctly predicted observation to the total observations. One may think that, if we have high accuracy then our model is best. Yes, accuracy is a great measure but only when you have symmetric datasets where values of false positive and false negatives are almost same. Therefore, you have to look at other parameters to evaluate the performance of your model. For example, a 0.75 or greater means our model is at least 75% accurate, which is very good.

Accuracy = TP+TN/TP+FP+FN+TN

**Precision (Specificity)** <br>
Precision is the ratio of correctly predicted positive observations over the total of both the correclty and incorrectly predicted positive observations. The question that this metric answer is of all the wires that are labeled as fraud, how many are actually fraud? High precision relates to the low false positive rate. For example, a 0.75 precision translates into a very precise prediction.

Precision = TP/TP+FP

**Recall (Sensitivity)** <br>
Recall is the ratio of correctly predicted positive observations over the total of the correclty predicted positive and incorrectly predicted negative observations. The question recall answers is of all the wires that are truly fraud, how many did we labeled? For example, a recall of 0.631 means that our model is good because it is above the threshold of 0.5.

Recall = TP/TP+FN

**F1 score** <br>
F1 Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. Intuitively it is not as easy to understand as accuracy, but F1 is usually more useful than accuracy, especially if you have an uneven class distribution. Accuracy works best if false positives and false negatives have similar cost. If the cost of false positives and false negatives are very different, it’s better to look at both Precision and Recall.

F1 Score = 2*(Recall * Precision) / (Recall + Precision)

#### ***North America***

In [ ]:
NN_103_output_df_y_train.shape # before SMOTE original y train

(70000, 1)

In [ ]:
NN_103_output_df_y_train[NN_103_output_df_y_train['FRAUD_LABEL']>=1].shape # # before SMOTE original y train filter

(549, 1)

In [ ]:
NN_103_sm_output_df_y_train.shape # original y train

(138902,)

In [ ]:
NN_103_sm_output_df_y_train[NN_103_sm_output_df_y_train>=1].shape # original y train prediction filter

(69451,)

In [ ]:
NN_103_sm_output_ann_y_pred_train.shape # ann y train level 1 predictions

(138902,)

In [ ]:
NN_103_sm_output_rf_y_pred_train.shape # random forest y train level 1 predictions

(138902,)

In [ ]:
NN_103_sm_output_ann_y_pred_train[NN_103_sm_output_ann_y_pred_train>=1].shape # ann y train level 1 prediction filter

(138902,)

In [ ]:
NN_103_sm_output_rf_y_pred_train[NN_103_sm_output_rf_y_pred_train>=1].shape  # random forest y train level 1 prediction filter

(98,)

In [ ]:
NN_103_sm_output_ensemble_x_train_df.shape # level 2 x train

(138902, 2)

In [ ]:
NN_103_sm_output_ensemble_x_train_df.shape # level 2 x train 

(138902, 2)

In [ ]:
NN_103_sm_output_df_y_train.shape # level 2 y train 

(138902,)

In [ ]:
NN_103_sm_output_ensemble_x_train_df[NN_103_sm_output_ensemble_x_train_df['ANN']>=1].shape # level 2 x train ANN filter

(138902, 2)

In [ ]:
NN_103_sm_output_ensemble_x_train_df[NN_103_sm_output_ensemble_x_train_df['RF']>=1].shape# level 2 x train RF filter

(98, 2)

In [ ]:
NN_103_sm_output_ensemble_x_test_df.shape # level 2 x test 

(59582, 2)

In [ ]:
NN_103_sm_output_ensemble_x_test_df[NN_103_sm_output_ensemble_x_test_df['ANN']>=1].shape # level 2 x test ANN filter

(59582, 2)

In [ ]:
NN_103_sm_output_ensemble_x_test_df[NN_103_sm_output_ensemble_x_test_df['RF']>=1].shape# level 2 x test RF filter

(55, 2)

In [ ]:
NN_103_sm_output_final_pred.shape # level 2 predictions

(59582,)

In [ ]:
NN_103_sm_output_final_pred[NN_103_sm_output_final_pred>=1].shape # level 2 predictions filter

(59527,)

#### MT103

In [ ]:
print(confusion_matrix(NN_103_sm_output_df_y_test,NN_103_sm_output_final_pred))
print(classification_report(NN_103_sm_output_df_y_test,NN_103_sm_output_final_pred))

[[   55 29736]
 [    0 29791]]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00     29791
           1       0.50      1.00      0.67     29791

    accuracy                           0.50     59582
   macro avg       0.75      0.50      0.34     59582
weighted avg       0.75      0.50      0.34     59582



#### MT202

In [ ]:
print(confusion_matrix(NN_202_sm_output_df_y_test,NN_202_sm_output_final_pred))
print(classification_report(NN_202_sm_output_df_y_test,NN_202_sm_output_final_pred))

[[   15 29784]
 [    0 29799]]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00     29799
           1       0.50      1.00      0.67     29799

    accuracy                           0.50     59598
   macro avg       0.75      0.50      0.33     59598
weighted avg       0.75      0.50      0.33     59598



#### ***Europe***

#### MT103

In [ ]:
print(confusion_matrix(EU_103_sm_output_df_y_test,EU_103_sm_output_final_pred))
print(classification_report(EU_103_sm_output_df_y_test,EU_103_sm_output_final_pred))

[[    9 29847]
 [    0 29856]]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00     29856
           1       0.50      1.00      0.67     29856

    accuracy                           0.50     59712
   macro avg       0.75      0.50      0.33     59712
weighted avg       0.75      0.50      0.33     59712



#### MT202

In [ ]:
EU_202_output_df_y_test[EU_202_output_df_y_test['FRAUD_LABEL']<1].shape # Label 0 in orginal split

(29878, 1)

In [ ]:
EU_202_output_df_y_test[EU_202_output_df_y_test['FRAUD_LABEL']>0].shape # Label 1 in orginal split

(122, 1)

In [ ]:
print(confusion_matrix(EU_202_sm_output_df_y_test,EU_202_sm_output_final_pred))
print(classification_report(EU_202_sm_output_df_y_test,EU_202_sm_output_final_pred))

[[27476  2402]
 [26962  2916]]
              precision    recall  f1-score   support

           0       0.50      0.92      0.65     29878
           1       0.55      0.10      0.17     29878

    accuracy                           0.51     59756
   macro avg       0.53      0.51      0.41     59756
weighted avg       0.53      0.51      0.41     59756



#### ***Asia***

ANN

In [ ]:
AS_103_sm_output_df_y_train[AS_103_sm_output_df_y_train>0].shape

(69226,)

In [ ]:
AS_103_sm_output_ann_y_pred_train[AS_103_sm_output_ann_y_pred_train>0].shape

(138452,)

In [ ]:
AS_103_sm_output_df_y_test[AS_103_sm_output_df_y_test>0].shape

(59260, 1)

In [ ]:
AS_103_sm_output_ann_y_pred_test[AS_103_sm_output_ann_y_pred_test>0].shape

(59260,)

RF

In [ ]:
AS_103_sm_output_df_y_train[AS_103_sm_output_df_y_train<1].shape

(69226,)

In [ ]:
AS_103_sm_output_rf_y_pred_train[AS_103_sm_output_rf_y_pred_train<1].shape

(125919,)

In [ ]:
AS_103_sm_output_df_y_test[AS_103_sm_output_df_y_test<1].shape

(59260, 1)

In [ ]:
AS_103_sm_output_rf_y_pred_test[AS_103_sm_output_rf_y_pred_test<1].shape

(53644,)

In [ ]:
AS_103_sm_output_df_y_train[AS_103_sm_output_df_y_train>0].shape

(69226,)

ANN

Train dataset

In [ ]:
AS_202_sm_output_df_y_train.shape # Train dataset size

(139280,)

In [ ]:
AS_202_sm_output_df_y_train[AS_202_sm_output_df_y_train>0].shape

(69640,)

In [ ]:
AS_202_sm_output_df_y_train[AS_202_sm_output_df_y_train<1].shape

(69640,)

In [ ]:
AS_202_sm_output_ann_y_pred_train[AS_202_sm_output_ann_y_pred_train>0].shape # Prediction from train dataset

(139280,)

In [ ]:
AS_202_sm_output_ann_y_pred_train[AS_202_sm_output_ann_y_pred_train<1].shape # Prediction from train dataset

(0,)

Test dataset

In [ ]:
AS_202_sm_output_df_y_test.shape # Test dataset size

(59676, 1)

In [ ]:
AS_202_sm_output_df_y_test[AS_202_sm_output_df_y_test['FRAUD_LABEL']>0].shape 

(29838, 1)

In [ ]:
AS_202_sm_output_df_y_test[AS_202_sm_output_df_y_test['FRAUD_LABEL']<1].shape 

(29838, 1)

In [ ]:
AS_202_sm_output_ann_y_pred_test[AS_202_sm_output_ann_y_pred_test>0].shape # Prediction from test dataset

(59676,)

In [ ]:
AS_202_sm_output_ann_y_pred_test[AS_202_sm_output_ann_y_pred_test<1].shape # Prediction from test dataset

(0,)

RF

Train dataset

In [ ]:
AS_202_sm_output_df_y_train.shape # Train dataset size

(139280,)

In [ ]:
AS_202_sm_output_df_y_train[AS_202_sm_output_df_y_train>0].shape

(69640,)

In [ ]:
AS_202_sm_output_df_y_train[AS_202_sm_output_df_y_train<1].shape

(69640,)

In [ ]:
AS_202_sm_output_rf_y_pred_train[AS_202_sm_output_rf_y_pred_train<1].shape # Prediction from train dataset

(7,)

In [ ]:
AS_202_sm_output_rf_y_pred_train[AS_202_sm_output_rf_y_pred_train>0].shape # Prediction from train dataset

(139273,)

Test dataset

In [ ]:
AS_202_sm_output_df_y_test.shape # Test dataset size

(59676, 1)

In [ ]:
AS_202_sm_output_df_y_test[AS_202_sm_output_df_y_test['FRAUD_LABEL']>0].shape

(29838, 1)

In [ ]:
AS_202_sm_output_df_y_test[AS_202_sm_output_df_y_test['FRAUD_LABEL']<1].shape

(29838, 1)

In [ ]:
AS_202_sm_output_rf_y_pred_test[AS_202_sm_output_rf_y_pred_test<1].shape # Prediction from train dataset

(0,)

In [ ]:
AS_202_sm_output_rf_y_pred_test[AS_202_sm_output_rf_y_pred_test>0].shape # Prediction from train dataset

(59676,)

In [ ]:
AS_202_sm_output_final_pred[AS_202_sm_output_final_pred<1].shape # Prediction from train dataset

(0,)

In [ ]:
AS_202_sm_output_final_pred[AS_202_sm_output_final_pred>0].shape # Prediction from train dataset

(59676,)

In [ ]:
AS_202_output_df_y_test.shape

(60000, 1)

In [ ]:
AS_202_output_df_y_test[AS_202_output_df_y_test['FRAUD_LABEL']<1].shape # Label 0 in orginal split

(59682, 1)

In [ ]:
AS_202_output_df_y_test[AS_202_output_df_y_test['FRAUD_LABEL']>0].shape # Label 1 in orginal split

(318, 1)

#### MT103

In [ ]:
print(confusion_matrix(AS_103_sm_output_df_y_test,AS_103_sm_output_final_pred))
print(classification_report(AS_103_sm_output_df_y_test,AS_103_sm_output_final_pred))

[[ 2762 26868]
 [ 2854 26776]]
              precision    recall  f1-score   support

           0       0.49      0.09      0.16     29630
           1       0.50      0.90      0.64     29630

    accuracy                           0.50     59260
   macro avg       0.50      0.50      0.40     59260
weighted avg       0.50      0.50      0.40     59260



#### MT202

In [ ]:
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
metrics.f1_score(AS_202_sm_output_df_y_test, AS_202_sm_output_final_pred, pos_label=1, labels=np.unique(AS_202_sm_output_final_pred))
print(confusion_matrix(AS_202_sm_output_df_y_test,AS_202_sm_output_final_pred))
print(classification_report(AS_202_sm_output_df_y_test,AS_202_sm_output_final_pred))

[[ 2281 27557]
 [ 2171 27667]]
              precision    recall  f1-score   support

           0       0.51      0.08      0.13     29838
           1       0.50      0.93      0.65     29838

    accuracy                           0.50     59676
   macro avg       0.51      0.50      0.39     59676
weighted avg       0.51      0.50      0.39     59676



## **Merge Final Predictions with X Test Data Sets**

In [ ]:
# Merge predictions to North America
NN_103_final_df = pd.DataFrame( np.column_stack((NN_103_sm_output_df_x_test.to_numpy(), NN_103_sm_output_final_pred)),  columns = ['TRANSACTION_MONTH','TRANSACTION_AMOUNT','FRAUD_LABEL'] )
NN_202_final_df = pd.DataFrame( np.column_stack((NN_202_sm_output_df_x_test.to_numpy(), NN_202_sm_output_final_pred)),  columns = ['TRANSACTION_MONTH','TRANSACTION_AMOUNT','FRAUD_LABEL'] )

# Merge predictions to Europe
EU_103_final_df = pd.DataFrame( np.column_stack((EU_103_sm_output_df_x_test.to_numpy(), EU_103_sm_output_final_pred)),  columns = ['TRANSACTION_MONTH','TRANSACTION_AMOUNT','FRAUD_LABEL'] )
EU_202_final_df = pd.DataFrame( np.column_stack((EU_202_sm_output_df_x_test.to_numpy(), EU_202_sm_output_final_pred)),  columns = ['TRANSACTION_MONTH','TRANSACTION_AMOUNT','FRAUD_LABEL'] )

# Merge predictions to Asia
AS_103_final_df = pd.DataFrame( np.column_stack((AS_103_sm_output_df_x_test.to_numpy(), AS_103_sm_output_final_pred)),  columns = ['TRANSACTION_MONTH','TRANSACTION_AMOUNT','FRAUD_LABEL'] )
AS_202_final_df = pd.DataFrame( np.column_stack((AS_202_sm_output_df_x_test.to_numpy(), AS_202_sm_output_final_pred)),  columns = ['TRANSACTION_MONTH','TRANSACTION_AMOUNT','FRAUD_LABEL'] )

## **Backup Final Results for Rule Tuning**

In [ ]:
NN_103_final_df.to_csv('/content/gdrive/MyDrive/DSC-680/NN_103_final_df.csv', index=False)
NN_202_final_df.to_csv('/content/gdrive/MyDrive/DSC-680/NN_202_final_df.csv', index=False)
EU_103_final_df.to_csv('/content/gdrive/MyDrive/DSC-680/EU_103_final_df.csv', index=False)
EU_202_final_df.to_csv('/content/gdrive/MyDrive/DSC-680/EU_202_final_df.csv', index=False)
AS_103_final_df.to_csv('/content/gdrive/MyDrive/DSC-680/AS_103_final_df.csv', index=False)
AS_202_final_df.to_csv('/content/gdrive/MyDrive/DSC-680/AS_202_final_df.csv', index=False)